In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
from utility_func import *
import scipy.io as sio
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import gmtime, strftime
import random
import os

# Device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
"""Generate H_p channel batch data"""
M = 1
N = 3
# total number of generated samples
num_generated_sample = 2
# H_p generation
H_p_testing_batch = channel_generation_batch_tensor(num_generated_sample, N, M)

In [3]:
N_Tx = 3
tau = 1
N_Rx = 2
Pol_Tx, Pol_Rx, Pol_Tx_blk, Pol_Rx_blk  = PingPong_MISO_polarization_init(N_Tx, N_Rx, tau)

In [4]:
def beamforming_loss(H, LSTMA, LSTMB, MLP_A_t, MLP_A_r, MLP_B_t, MLP_B_r, MLP_A, MLP_B, NN_parameters, train_parameters):
    'Get parameter values'
    (_, _, hidden_sizeA, hidden_sizeB) = NN_parameters
    (batch_size, N, M, L, N0) = train_parameters
    # hidden_sizeA = parameters['A_hidden']
    # hidden_sizeB = parameters['B_hidden']
    # batch_size = parameters['batch_size']
    # N = parameters['Tx_size']
    # M = parameters['Rx_size']
    # L = parameters['numofRounds']
    # N0 = parameters['noisePower']

    """Initialize the hidden unit and the cell state"""
    h_A = torch.zeros(([batch_size, hidden_sizeA]))
    c_A = torch.zeros(([batch_size, hidden_sizeA]))
    h_B = torch.zeros(([batch_size, hidden_sizeB]))
    c_B = torch.zeros(([batch_size, hidden_sizeB]))

    """Start the ping pong pilot rounds"""
    for t in range(L):
        if t == 0:
            """Initialize parameters for the first pilot stage"""
            # Initialize polarization vectors
            _, _, Pol_A_t_blk, Pol_B_r_blk = PingPong_MISO_polarization_init(N,M,batch_size)
            Pol_A_t_blk = Pol_A_t_blk.to(torch.complex64)
            Pol_A_r_blk = Pol_A_t_blk
            Pol_A_r_blk_T = torch.transpose(Pol_A_r_blk, 1, 2)
            Pol_B_r_blk_T = torch.transpose(Pol_B_r_blk, 1, 2).to(torch.complex64)
            # Initialize transmit vector from Agent A
            W_A_t_real = torch.randn((batch_size, 1, N))
            W_A_t_imag = torch.randn((batch_size, 1, N))
            W_A_t = torch.complex(W_A_t_real, W_A_t_imag)
            W_A_t = W_A_t / torch.norm(W_A_t, dim=2, keepdim=True)
            W_A_t = torch.transpose(W_A_t, 1, 2)
            # Initialize receive vector from Agent A 
            W_A_r_real = torch.randn((batch_size, 1, N))
            W_A_r_imag = torch.randn((batch_size, 1, N))
            W_A_r = torch.complex(W_A_r_real, W_A_r_imag)
            W_A_r = W_A_r / torch.norm(W_A_r, dim=2, keepdim=True)
            W_A_r_Her = W_A_r.conj()

        """Agent B observes the measurement"""
        y_noiseless_B = Pol_B_r_blk_T @ H @ Pol_A_t_blk @ W_A_t
        noise_sqrt = torch.sqrt(N0)
        noise_real  = torch.randn((batch_size, 1, 1))
        noise_imag = torch.randn((batch_size, 1, 1))
        noise = torch.complex(noise_real, noise_imag)
        sqrt_2_tensor = torch.sqrt(torch.tensor(2.0))
        noise = noise / sqrt_2_tensor.unsqueeze(-1).unsqueeze(-1)
        noise = noise * noise_sqrt
        y_B = y_noiseless_B + noise
        y_real_B = torch.squeeze(torch.concatenate([y_B.real, y_B.imag], axis=2))
        print(y_real_B)

        """Agent B design the next receive polarization"""
        h_B, c_B = LSTMB((y_real_B, h_B, c_B))
        output_B_r = MLP_B_r(h_B)
        angle_B_r = torch.sigmoid(output_B_r) * torch.pi
        Pol_B_r = torch.hstack((torch.cos(angle_B_r), torch.sin(angle_B_r)))
        Pol_B_r_blk = vec2block_diag(Pol_B_r)
        Pol_B_r_blk_T = torch.transpose(Pol_B_r_blk, 1, 2)

        """Agent B design the next transmit polarization"""
        output_B_t = MLP_B_t(h_B)
        angle_B_t = torch.sigmoid(output_B_t) * torch.pi
        Pol_B_t = torch.hstack((torch.cos(angle_B_t), torch.sin(angle_B_t)))
        Pol_B_t_blk = vec2block_diag(Pol_B_t)

        """Agent A observes the measurement"""
        y_noiseless_A = W_A_r_Her @ Pol_A_r_blk_T @ H @ Pol_B_t_blk
        noise_sqrt = torch.sqrt(N0)
        noise_real  = torch.randn((batch_size, 1, 1))
        noise_imag = torch.randn((batch_size, 1, 1))
        noise = torch.complex(noise_real, noise_imag)
        sqrt_2_tensor = torch.sqrt(torch.tensor(2.0))
        noise = noise / sqrt_2_tensor.unsqueeze(-1).unsqueeze(-1)
        noise = noise * noise_sqrt
        y_A = y_noiseless_A + noise
        y_real_A = torch.squeeze(torch.concatenate([y_A.real, y_A.imag], axis=2))

        """Agent A design the next receive polarization and beamformer"""
        h_A, c_A = LSTMA((y_real_A, h_A, c_A))
        output_A_r = MLP_A_r(h_A)
        angle_A_r = torch.sigmoid(output_A_r[:,:,:N]) * torch.pi
        Pol_A_r = torch.hstack((torch.cos(angle_A_r), torch.sin(angle_A_r)))
        Pol_A_r_blk = vec2block_diag(Pol_A_r)
        Pol_A_r_blk_T = torch.transpose(Pol_A_r_blk,1,2)

        W_A_r = output_A_r[:,:,N:]
        W_A_r_norm = torch.norm(W_A_r, dim=2, keepdim=True)
        W_A_r = W_A_r/W_A_r_norm
        W_A_r = torch.complex(W_A_r[:,:,:N],W_A_r[:,:,N:])
        W_A_r_Her = W_A_r.conj()

        """Agent A design the next transmit polarization and beamformer"""
        output_A_t = MLP_A_t(h_A)
        angle_A_t = torch.sigmoid(output_A_t[:,:,:N]) * torch.pi
        Pol_A_t = torch.hstack((torch.cos(angle_A_t), torch.sin(angle_A_t)))
        Pol_A_t_blk = vec2block_diag(Pol_A_t)

        W_A_t = output_A_t[:,:,N:]
        W_A_t_norm = torch.norm(W_A_t, dim=2, keepdim=True)
        W_A_t = W_A_t/W_A_t_norm
        W_A_t = torch.complex(W_A_t[:,:,:N],W_A_t[:,:,N:])
        W_A_t = torch.transpose(W_A_r, 1, 2)

    'Calculate final optimal outputs'
    # Agent A side 
    output_A_t = MLP_A(c_A)
    angle_A_t = torch.sigmoid(output_A_t[:,:,:N]) * torch.pi
    Pol_A_t = torch.hstack((torch.cos(angle_A_t), torch.sin(angle_A_t)))
    Pol_A_t_blk = vec2block_diag(Pol_A_t)
    W_A_t = output_A_t[:,:,N:]
    W_A_t_norm = torch.norm(W_A_t, dim=2, keepdim=True)
    W_A_t = W_A_t/W_A_t_norm
    W_A_t = torch.complex(W_A_t[:,:,:N],W_A_t[:,:,N:])

    # Agent B side
    output_B_r = MLP_B(c_B)
    angle_B_r = torch.sigmoid(output_B_r) * torch.pi
    Pol_B_r = torch.hstack((torch.cos(angle_B_r), torch.sin(angle_B_r)))
    Pol_B_r_blk = vec2block_diag(Pol_B_r)
    Pol_B_r_blk_T = torch.transpose(Pol_B_r_blk,1,2)

    bf_gain = torch.sum(torch.abs((Pol_B_r_blk_T @ H @ Pol_A_t_blk @ W_A_t)**2))
    bf_gain_loss = -bf_gain
    
    return bf_gain_loss

In [5]:
"""Initialize the hidden unit and the cell state"""
batch_size = 1
hidden_sizeA = 2
hidden_sizeB = 2
L = 3
H = H_p_testing_batch
N0 = torch.sqrt(torch.tensor(10**(0/10)))
h_A = torch.zeros(([batch_size, hidden_sizeA]))
c_A = torch.zeros(([batch_size, hidden_sizeA]))
h_B = torch.zeros(([batch_size, hidden_sizeB]))
c_B = torch.zeros(([batch_size, hidden_sizeB]))

"""Start the ping pong pilot rounds"""
for t in range(L):
    if t == 0:
        """Initialize parameters for the first pilot stage"""
        # Initialize polarization vectors
        _, _, Pol_A_t_blk, Pol_B_r_blk = PingPong_MISO_polarization_init(N,M,batch_size)
        Pol_A_t_blk = Pol_A_t_blk.to(torch.complex64)
        Pol_A_r_blk = Pol_A_t_blk
        Pol_A_r_blk_T = torch.transpose(Pol_A_r_blk, 1, 2)
        Pol_B_r_blk_T = torch.transpose(Pol_B_r_blk, 1, 2).to(torch.complex64)
        # Initialize transmit vector from Agent A
        W_A_t_real = torch.randn((batch_size, 1, N))
        W_A_t_imag = torch.randn((batch_size, 1, N))
        W_A_t = torch.complex(W_A_t_real, W_A_t_imag)
        W_A_t = W_A_t / torch.norm(W_A_t, dim=2, keepdim=True)
        W_A_t = torch.transpose(W_A_t, 1, 2)
        # Initialize receive vector from Agent A 
        W_A_r_real = torch.randn((batch_size, 1, N))
        W_A_r_imag = torch.randn((batch_size, 1, N))
        W_A_r = torch.complex(W_A_r_real, W_A_r_imag)
        W_A_r = W_A_r / torch.norm(W_A_r, dim=2, keepdim=True)
        W_A_r_Her = W_A_r.conj()

    """Agent B observes the measurement"""
    y_noiseless_B = Pol_B_r_blk_T @ H @ Pol_A_t_blk @ W_A_t
    noise_sqrt = torch.sqrt(N0)
    noise_real  = torch.randn((batch_size, 1, 1))
    noise_imag = torch.randn((batch_size, 1, 1))
    noise = torch.complex(noise_real, noise_imag)
    sqrt_2_tensor = torch.sqrt(torch.tensor(2.0))
    noise = noise / sqrt_2_tensor.unsqueeze(-1).unsqueeze(-1)
    noise = noise * noise_sqrt
    y_B = y_noiseless_B + noise
    y_real_B = torch.squeeze(torch.concatenate([y_B.real, y_B.imag], axis=2))
    print(y_real_B)

tensor([[-0.0159,  1.4413],
        [ 1.4019,  0.8818]])
tensor([[-0.0794,  0.8395],
        [ 1.3384,  0.2800]])
tensor([[ 0.6775,  0.4724],
        [ 2.0953, -0.0871]])


In [6]:
print(H.shape)
print(Pol_B_r_blk_T.shape)
print(Pol_A_t_blk.shape)
print(W_A_t.shape)

torch.Size([2, 2, 6])
torch.Size([1, 1, 2])
torch.Size([1, 6, 3])
torch.Size([1, 3, 1])


In [7]:
class MLPBlock(nn.Module):
    def __init__(self, num_layers, dims):
        super(MLPBlock, self).__init__()

        layers = []
        for i in range(num_layers - 2):
            print(i)
            layers.append(nn.Linear(dims[i], dims[i + 1]))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(dims[i + 1]))

        layers.append(nn.Linear(dims[-2], dims[-1]))
        self.mlp = nn.Sequential(*layers)

    def forward(self, inputs):
        return self.mlp(inputs)

# Assuming MLP_B_r has three layers (adjust accordingly)
mlp_b_r_dims = [256, 512, 1]
mlp_b_r = MLPBlock(len(mlp_b_r_dims), mlp_b_r_dims)

# Set the model to evaluation mode
mlp_b_r.eval()

# Forward pass with a dummy input
dummy_input = torch.randn((1024, 256))  # Adjust the size accordingly
output = mlp_b_r(dummy_input)

# Check the dimensions of the output
print("Output shape:", output.shape)

0
Output shape: torch.Size([1024, 1])


In [8]:
Pol_Tx, Pol_Rx, Pol_Tx_blk, Pol_Rx_blk = PingPong_MISO_polarization_init(64, 1, 10)
print(Pol_Tx.shape)
print(Pol_Rx_blk.shape)
print(torch.transpose(Pol_Rx_blk,1,2).shape)


torch.Size([10, 2, 64])
torch.Size([10, 2, 1])
torch.Size([10, 1, 2])


In [9]:
A = torch.rand(1024, 192)
A_norm = torch.norm(A, dim=1,keepdim=True)
A = A/A_norm
print(A.shape)
print(torch.sum(torch.abs(A[0])**2))
# B = A[:,:2]
# print(A.shape)
# print(B.shape)

torch.Size([1024, 192])
tensor(1.)


In [10]:
W_A_t_real = torch.randn((3, 1, N))
W_A_t_imag = torch.randn((3, 1, N))
W_A_t = torch.complex(W_A_t_real, W_A_t_imag)
print(W_A_t)
W_A_t_normalizing = torch.norm(W_A_t, dim=2, keepdim=True)
print(W_A_t_normalizing)
W_A_t = W_A_t / torch.norm(W_A_t, dim=2, keepdim=True)
print(torch.sum(torch.abs(W_A_t[2])**2))

tensor([[[ 1.4393-0.7210j, -0.7183+2.1844j, -1.9401-0.8702j]],

        [[-0.6868-0.3108j,  0.1676-0.4853j,  1.4804+0.1165j]],

        [[-1.9918+1.9993j, -0.7912-0.6580j, -0.6792-0.7195j]]])
tensor([[[3.5214]],

        [[1.7427]],

        [[3.1627]]])
tensor(1.0000)


In [11]:
def TwoWay_Pilots(H, batch_size, parameters):
    'Get parameter values'
    N, M, L, N0 = parameters 
    y_real_B = torch.zeros((batch_size,2,L))
    y_real_A = torch.zeros((batch_size,N,2,L))

    """Start the ping pong pilot rounds"""
    H_her = torch.transpose(H.conj(), 1 ,2)
    for t in range(L):
        """Receiver observes L Pilots"""
        # Initialize polarization vectors at rx
        _, _, Pol_A_t_blk, Pol_B_r_blk = PingPong_MISO_polarization_init(N,M,batch_size)    
        Pol_A_t_blk = Pol_A_t_blk.to(torch.complex64)
        Pol_B_r_blk_T = torch.transpose(Pol_B_r_blk, 1, 2).to(torch.complex64)

        # Initialize transmit vector from Agent A
        W_A_t_real = torch.randn((batch_size, 1, N))
        W_A_t_imag = torch.randn((batch_size, 1, N))
        W_A_t = torch.complex(W_A_t_real, W_A_t_imag)
        W_A_t = W_A_t / torch.norm(W_A_t, dim=2, keepdim=True)
        W_A_t = torch.transpose(W_A_t, 1, 2)

        # Making receiver pilots
        Heff = Pol_B_r_blk_T @ H @ Pol_A_t_blk
        y_noiseless_B = Heff @ W_A_t
        noise_sqrt = torch.sqrt(N0)
        noise_real  = torch.randn((batch_size, 1, 1))
        noise_imag = torch.randn((batch_size, 1, 1))
        noise = torch.complex(noise_real, noise_imag)
        sqrt_2_tensor = torch.sqrt(torch.tensor(2.0))
        noise = noise / sqrt_2_tensor.unsqueeze(-1).unsqueeze(-1)
        noise = noise * noise_sqrt
        y_B = y_noiseless_B + noise
        # print(y_B)
        y_real_B[:,:,t] = torch.squeeze(torch.concatenate([y_B.real, y_B.imag], axis=2))

        """Transmitter observes L Pilots"""
        # Initialize polarization vectors at tx
        _, _, Pol_A_r_blk, Pol_B_t_blk = PingPong_MISO_polarization_init(N,M,batch_size)
        Pol_B_t_blk = Pol_B_t_blk.to(torch.complex64)
        Pol_A_r_blk_T = torch.transpose(Pol_A_r_blk, 1, 2).to(torch.complex64)

        # Initialize receive vector from tx 
        W_A_r_real = torch.randn((batch_size, 1, N))
        W_A_r_imag = torch.randn((batch_size, 1, N))
        W_A_r = torch.complex(W_A_r_real, W_A_r_imag)
        W_A_r = W_A_r / torch.norm(W_A_r, dim=2, keepdim=True)
        W_A_r_Her = W_A_r.conj()

        # Making receiver pilots
        H_eff_her = Pol_A_r_blk_T @ H_her @ Pol_B_t_blk
        # print("H_eff_her")
        # print(H_eff_her.shape)
        y_noiseless_A = H_eff_her
        noise_sqrt = torch.sqrt(N0)
        noise_real  = torch.randn((batch_size, N, 1))
        noise_imag = torch.randn((batch_size, N, 1))
        noise = torch.complex(noise_real, noise_imag)
        sqrt_2_tensor = torch.sqrt(torch.tensor(2.0))
        noise = noise / sqrt_2_tensor.unsqueeze(-1).unsqueeze(-1)
        noise = noise * noise_sqrt
        y_A = y_noiseless_A + noise
        # print(y_A)
        y_real_A[:,:,:,t] = torch.squeeze(torch.concatenate([y_A.real, y_A.imag], axis=2))

    return y_real_B, y_real_A

In [12]:
"""Generate H_p channel batch data"""
N = 4
M = 1
L = 2
N0 = torch.tensor(1)
batch_size = 3
# total number of generated samples
num_generated_testing_sample = batch_size
# H_p generation
H_p_testing_batch = channel_generation_batch_tensor(num_generated_testing_sample, N, M)
H = H_p_testing_batch
parameters = (N,M,L,N0) 

In [13]:
y_real_B, y_real_A =TwoWay_Pilots(H, batch_size, parameters)
# print(y_real_B)
Y_A_real = torch.transpose(y_real_A[:,:,0,:],1,2).reshape(batch_size,-1)
Y_A_imag = torch.transpose(y_real_A[:,:,1,:],1,2).reshape(batch_size,-1)
print("Y_A_real")
print(Y_A_real)
print("Y_A_imag")
print(Y_A_imag)
print(torch.concatenate([Y_A_real, Y_A_imag], axis=1))
# print(y_real_A)

Y_A_real
tensor([[ 1.6275,  1.4283,  0.8873, -0.6026,  0.0258,  0.8300,  0.8604, -0.2669],
        [ 0.1683,  0.7540,  1.7726, -0.9585, -0.6287,  0.2489, -0.1854,  1.3520],
        [-0.4511, -0.8777,  0.4653,  1.4556,  0.0073,  0.1163,  1.3161,  0.1265]])
Y_A_imag
tensor([[-7.3520e-02,  4.4778e-01, -3.8339e-01, -2.9087e-01,  8.5497e-01,
         -3.5764e+00,  6.9475e-01, -1.1451e+00],
        [ 9.0620e-01, -9.0245e-01, -7.8877e-02,  2.6410e-01,  5.3711e-01,
          9.7309e-01, -1.0698e+00, -6.8520e-01],
        [-1.1876e+00,  1.8883e+00, -9.8804e-02,  8.1048e-01,  1.6334e-03,
          8.9639e-01, -2.7595e-01, -9.8066e-01]])
tensor([[ 1.6275e+00,  1.4283e+00,  8.8734e-01, -6.0262e-01,  2.5797e-02,
          8.2995e-01,  8.6041e-01, -2.6692e-01, -7.3520e-02,  4.4778e-01,
         -3.8339e-01, -2.9087e-01,  8.5497e-01, -3.5764e+00,  6.9475e-01,
         -1.1451e+00],
        [ 1.6834e-01,  7.5396e-01,  1.7726e+00, -9.5854e-01, -6.2867e-01,
          2.4891e-01, -1.8537e-01,  1.3520e+00

In [14]:
Pol_Tx, Pol_Rx, Pol_Tx_blk, Pol_Rx_blk = MISO_polarization_pilot_tensor(3,1,2)
print(torch.transpose(Pol_Tx_blk,1,2))

tensor([[[-0.9517+0.j,  0.3070+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  0.5908+0.j, -0.8068+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.8304+0.j,  0.5571+0.j]],

        [[ 0.9641+0.j,  0.2657+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  0.1495+0.j,  0.9888+0.j,  0.0000+0.j,  0.0000+0.j],
         [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j, -0.1139+0.j,  0.9935+0.j]]])


In [69]:
L = 5
N_Tx = 2
N_Rx = 1

In [72]:
def MISO_polarization_pilot_tensor_fixed(N_Tx, N_Rx, tau):
    """
    This function generates polarization vectors
    in the case where receiver polarization is fixed.
    """
    # initialization 
    Pol_BS = torch.zeros((tau, 2, N_Tx))
    Pol_UE = torch.zeros((tau, 2, N_Rx))

    # generate random angles
    theta_BS_t = torch.randn(1, N_Tx) * torch.pi; theta_BS_t = theta_BS_t.expand(L, 1, N_Tx)
    repeat_index = L//4; truncate_index = 4-L%4
    theta_BS_r = torch.randn(4, 1, N_Tx); theta_BS_r = theta_BS_r.repeat(repeat_index+1,1,1)[:-truncate_index]
    theta_UE_t = torch.randn(1, N_Rx) * torch.pi; theta_UE_t = theta_UE_t.expand(L,1,N_Rx)
    theta_UE_r = torch.randn(1, N_Rx) * torch.pi; theta_UE_r = theta_UE_r.expand(L,1,N_Rx)

    # vstack stacks ups cos(theta_tx) and sin(theta_tx) as column vectors
    Pol_BS_t = torch.hstack((torch.cos(theta_BS_t), torch.sin(theta_BS_t)))
    Pol_BS_r = torch.hstack((torch.cos(theta_BS_r), torch.sin(theta_BS_r)))
    Pol_UE_t = torch.hstack((torch.cos(theta_UE_t), torch.sin(theta_UE_t)))
    Pol_UE_r = torch.hstack((torch.cos(theta_UE_r), torch.sin(theta_UE_r)))

    Pol_BS_t_blk = vec2block_diag(Pol_BS_t).to(torch.complex64)
    Pol_BS_r_blk = vec2block_diag(Pol_BS_r).to(torch.complex64)
    Pol_UE_t_blk = vec2block_diag(Pol_UE_t).to(torch.complex64)
    Pol_UE_r_blk = vec2block_diag(Pol_UE_r).to(torch.complex64)

    return Pol_BS_t, Pol_BS_r, Pol_UE_t, Pol_UE_r, Pol_BS_t_blk, Pol_BS_r_blk, Pol_UE_t_blk, Pol_UE_r_blk

In [73]:
Pol_BS_t, Pol_BS_r, Pol_UE_t, Pol_UE_r = MISO_polarization_pilot_tensor_fixed(N_Tx, N_Rx, L)
print(Pol_BS_t)
print(Pol_BS_r)
print(Pol_UE_t)
print(Pol_UE_r)



tensor([[[ 0.9987, -0.9153],
         [ 0.0508, -0.4028]],

        [[ 0.9987, -0.9153],
         [ 0.0508, -0.4028]],

        [[ 0.9987, -0.9153],
         [ 0.0508, -0.4028]],

        [[ 0.9987, -0.9153],
         [ 0.0508, -0.4028]],

        [[ 0.9987, -0.9153],
         [ 0.0508, -0.4028]]])
tensor([[[ 0.8968,  0.7210],
         [ 0.4425,  0.6929]],

        [[ 0.9991,  0.6783],
         [ 0.0428, -0.7348]],

        [[ 0.8761,  0.5291],
         [-0.4821, -0.8485]],

        [[ 0.1702,  0.9913],
         [ 0.9854,  0.1315]],

        [[ 0.8968,  0.7210],
         [ 0.4425,  0.6929]]])
tensor([[[ 0.8087],
         [-0.5882]],

        [[ 0.8087],
         [-0.5882]],

        [[ 0.8087],
         [-0.5882]],

        [[ 0.8087],
         [-0.5882]],

        [[ 0.8087],
         [-0.5882]]])
tensor([[[-0.3500],
         [ 0.9368]],

        [[-0.3500],
         [ 0.9368]],

        [[-0.3500],
         [ 0.9368]],

        [[-0.3500],
         [ 0.9368]],

        [[-0.3500],
  

In [65]:
L = 10
theta_BS_r = torch.randn(4, 1, N_Tx)
print(theta_BS_r)
repeat_index = L//4
print(repeat_index)
truncate_index = 4-L%4
print(truncate_index)
theta_BS_r = theta_BS_r.repeat(repeat_index+1,1,1)[:-truncate_index]
print(theta_BS_r)

tensor([[[ 0.1686,  0.1223]],

        [[ 2.7955,  0.0959]],

        [[ 1.1260,  0.8322]],

        [[-0.6500, -0.2456]]])
2
2
tensor([[[ 0.1686,  0.1223]],

        [[ 2.7955,  0.0959]],

        [[ 1.1260,  0.8322]],

        [[-0.6500, -0.2456]],

        [[ 0.1686,  0.1223]],

        [[ 2.7955,  0.0959]],

        [[ 1.1260,  0.8322]],

        [[-0.6500, -0.2456]],

        [[ 0.1686,  0.1223]],

        [[ 2.7955,  0.0959]]])


In [66]:
print(4 %4)

0
